In [3]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# Redis URL 가져오기
REDIS_URL = os.getenv('REDIS_URL')

print(f"Redis URL: {REDIS_URL}")


Redis URL: redis://:dlp@localhost:6379


In [4]:
import os
from langchain_redis import RedisVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from dotenv import load_dotenv

load_dotenv()

# Redis URL 설정
REDIS_URL = os.getenv('REDIS_URL')

# 벡터 스토어 초기화
embeddings = OpenAIEmbeddings()
vector_store = RedisVectorStore(embeddings, redis_url=REDIS_URL)

# 문서 추가
docs = [
    Document(page_content="Redis는 강력한 인메모리 데이터베이스입니다."),
    Document(page_content="LangChain은 AI 애플리케이션 개발을 위한 프레임워크입니다.")
]
vector_store.add_documents(docs)

# 유사도 검색 실행
query = "Redis란 무엇인가?"
results = vector_store.similarity_search(query)

# 검색 결과 출력
print(f"'{query}'에 대한 검색 결과:")
for doc in results:
    print(f"- {doc.page_content}")


'Redis란 무엇인가?'에 대한 검색 결과:
- Redis는 강력한 인메모리 데이터베이스입니다.
- LangChain은 AI 애플리케이션 개발을 위한 프레임워크입니다.


In [5]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_redis import RedisChatMessageHistory

In [6]:
# Initialize RedisChatMessageHistory
history = RedisChatMessageHistory(session_id="user_123", redis_url=REDIS_URL)

# Add messages to the history
history.add_user_message("안녕 봇!")
history.add_ai_message("그래 안녕...?")

# Retrieve messages
print("Chat History:")
for message in history.messages:
    print(f"{type(message).__name__}: {message.content}")

Chat History:
HumanMessage: 안녕 봇!
AIMessage: 그래 안녕...?


In [7]:
from getpass import getpass

# Check if OPENAI_API_KEY is already set in the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("OpenAI API key not found in environment variables.")
    openai_api_key = getpass("Please enter your OpenAI API key: ")

    # Set the API key for the current session
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OpenAI API key has been set for this session.")
else:
    print("OpenAI API key found in environment variables.")

OpenAI API key found in environment variables.


In [8]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# Initialize the language model
llm = ChatOpenAI()

# Create the conversational chain
chain = prompt | llm


# Function to get or create a RedisChatMessageHistory instance
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, redis_url=REDIS_URL)


# Create a runnable with message history
chain_with_history = RunnableWithMessageHistory(
    chain, get_redis_history, input_messages_key="input", history_messages_key="history"
)

# Use the chain in a conversation
response1 = chain_with_history.invoke(
    {"input": "Hi, my name is Alice."},
    config={"configurable": {"session_id": "alice_123"}},
)
print("AI Response 1:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What's my name?"}, config={"configurable": {"session_id": "alice_123"}}
)
print("AI Response 2:", response2.content)

AI Response 1: Hello, Alice! How can I assist you today?
AI Response 2: Your name is Alice.


In [9]:
# Initialize with custom Redis configuration
custom_history = RedisChatMessageHistory(
    "user_456",
    redis_url=REDIS_URL,
    key_prefix="custom_prefix:",
    ttl=3600,  # Set TTL to 1 hour
    index_name="custom_index",
)

custom_history.add_user_message("This is a message with custom configuration.")
print("Custom History:", custom_history.messages)

Custom History: [HumanMessage(content='This is a message with custom configuration.', additional_kwargs={}, response_metadata={})]


In [10]:
# Add more messages
history.add_user_message("Tell me about artificial intelligence.")
history.add_ai_message(
    "Artificial Intelligence (AI) is a branch of computer science..."
)

# Search for messages containing a specific term
search_results = history.search_messages("artificial intelligence")
print("Search Results:")
for result in search_results:
    print(f"{result['type']}: {result['content'][:50]}...")

Search Results:
human: Tell me about artificial intelligence....
ai: Artificial Intelligence (AI) is a branch of comput...


In [11]:
# Clear the chat history
history.clear()
print("Messages after clearing:", history.messages)

Messages after clearing: []


In [12]:
import os
import uuid
from langchain_redis import RedisChatMessageHistory
from langchain.chains import ConversationChain
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# Redis URL 및 OpenAI API 키 로드
REDIS_URL = os.getenv("REDIS_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI API 키 설정

# 고유한 session_id 생성 (예: UUID 사용)
session_id = str(uuid.uuid4())  # UUID로 고유한 session_id 생성

# Redis에 연결하여 메시지 이력 관리 객체 생성
message_history = RedisChatMessageHistory(redis_url=REDIS_URL, session_id=session_id)

# OpenAI 모델 설정 (프롬프트에 따라 동적으로 응답을 생성)
llm = OpenAI(temperature=0.7)

# ConversationBufferMemory를 사용하여 메모리 처리
memory = ConversationBufferMemory(memory_key="history")

# 대화 체인 초기화 (Redis 메시지 기록을 사용하는 ConversationChain)
conversation = ConversationChain(llm=llm, memory=memory)

# 사용자 입력을 받아 응답을 생성하는 함수
def chat_with_bot(user_input):
    # 챗봇에게 입력을 주고, 대답을 받기
    response = conversation.predict(input=user_input)
    return response

# 단기 기억 기능을 활용한 대화 예시
print("안녕하세요! 무엇을 도와드릴까요?")

# 대화 예시: 사용자가 처음 질문을 했을 때
while True:
    user_input = input("나: ")

    # 종료 조건 설정
    if user_input.lower() in ['exit', 'quit', '종료']:
        print("챗봇을 종료합니다. 감사합니다!")
        break
    
    # 단기 기억을 활용한 대화 흐름
    response = chat_with_bot(user_input)
    print(f"챗봇: {response}")

    # 대화 이력 출력 (단기 기억 상태)
    print("\n--- 현재 단기 기억 ---")
    print(memory.load_memory_variables({}))
    print("----------------------")


C:\Users\20111\AppData\Local\Temp\ipykernel_14980\1753710674.py:28: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history")
C:\Users\20111\AppData\Local\Temp\ipykernel_14980\1753710674.py:31: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory)


안녕하세요! 무엇을 도와드릴까요?
챗봇:  Hello there! How may I assist you?

--- 현재 단기 기억 ---
{'history': 'Human: \nAI:  Hello there! How may I assist you?'}
----------------------
챗봇:  Sure, I can definitely help you with that! Based on your location and the time of day, I would recommend checking out a nearby Italian restaurant called "Pasta Paradise." They have a great selection of pasta dishes and their dinner specials include a delicious pesto chicken dish. Would you like me to provide directions or the restaurant's contact information for you?

--- 현재 단기 기억 ---
{'history': 'Human: \nAI:  Hello there! How may I assist you?\nHuman: 추천해줄 저녁메뉴\nAI:  Sure, I can definitely help you with that! Based on your location and the time of day, I would recommend checking out a nearby Italian restaurant called "Pasta Paradise." They have a great selection of pasta dishes and their dinner specials include a delicious pesto chicken dish. Would you like me to provide directions or the restaurant\'s contact informa

In [13]:
## 리밋(10개) 제한 (v)
## 특정 시간이 지나면 redis 저장소에서 자동 삭제 (v) (20초)
## 세션(redis 서버(docker 서버))가 닫히면 자동 삭제 (v) (이거 실행 잘못하면 container 다시 설치하고 다시 비밀번호 셋팅해야함...<<이 사람이 아까 녹은 이유)

import os
import redis
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# .env 파일 로드
load_dotenv()

# 환경 변수 로드
REDIS_URL = os.getenv('REDIS_URL', 'redis://localhost:6379')  # 기본값 설정
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD', '')
REDIS_PORT = os.getenv('REDIS_PORT', 6379)
REDIS_WEB_PORT = os.getenv('REDIS_WEB_PORT', 8001)

print(f"Redis URL: {REDIS_URL}")
print(f"Redis Password: {REDIS_PASSWORD}")
print(f"Redis Port: {REDIS_PORT}")
print(f"Redis Web Port: {REDIS_WEB_PORT}")

redis_client = redis.StrictRedis.from_url(REDIS_URL, decode_responses=True)

# 단기 기억 크기 설정
MAX_MEMORY_SIZE = 3  # 최대 저장할 메시지 수
TTL = 20  # 단기 기억 유효 시간 (20초)

# 사용자 세션 ID (여기서는 예시로 고정값 사용, 실제 서비스에서는 유저별 고유 ID 생성 필요)
session_id = "unique_session_id"
short_term_memory_key = f"user_short_term_memory:{session_id}"

# OpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# LangChain 메모리 설정
conversation_memory = ConversationBufferMemory(memory_key="history")

# ConversationChain 생성
conversation = ConversationChain(
    llm=llm,
    memory=conversation_memory
)

# Redis에 단기 기억 저장 함수
def add_to_short_term_memory(query, response):
    message = f"Q: {query}\nA: {response}"
    redis_client.lpush(short_term_memory_key, message)
    redis_client.expire(short_term_memory_key, TTL)  # TTL 설정
    redis_client.ltrim(short_term_memory_key, 0, MAX_MEMORY_SIZE - 1)  # 최대 크기 유지

# Redis에서 단기 기억 로드 함수
def load_short_term_memory():
    return redis_client.lrange(short_term_memory_key, 0, -1)

# LangChain 메모리와 Redis 동기화 함수
def sync_memory():
    messages = load_short_term_memory()
    if messages:
        conversation_memory.clear()  # 기존 메모리 초기화
        for message in reversed(messages):  # 최신 메시지가 마지막에 오도록
            q, a = message.split("\nA: ")
            q = q.replace("Q: ", "")
            conversation_memory.chat_memory.add_user_message(q)
            conversation_memory.chat_memory.add_ai_message(a)

# 대화 함수
def chat(query):
    sync_memory()  # 단기 기억 동기화
    try:
        response = conversation.predict(input=query)
        add_to_short_term_memory(query, response)  # Redis에 대화 내용 저장
        return response
    except Exception as e:
        print(f"오류 발생: {e}")
        return "오류가 발생했습니다. 다시 시도해주세요."

# 예시 대화
if __name__ == "__main__":
    # 여러 질문을 던져 단기 기억이 10개를 초과하도록 테스트
    queries = [
        "Redis란 무엇인가요?",
        "LangChain이 무엇인가요?",
        "Python은 무엇에 사용되나요?",
        "인공지능의 주요 응용 분야는 무엇인가요?",
        "Redis의 TTL 기능이란 무엇인가요?",
        "LangChain의 Memory는 어떤 역할을 하나요?",
    ]

    # 각 질문에 대한 답변 출력
    for i, query in enumerate(queries, start=1):
        print(f"질문 {i}: {query}")
        response = chat(query)
        print(f"답변 {i}: {response}\n")

    # 단기 기억 내용 확인
    short_term_memory = load_short_term_memory()
    print("단기 기억 내용:")
    for i, memory in enumerate(short_term_memory, start=1):
        print(f"{i}: {memory}")



Redis URL: redis://:dlp@localhost:6379
Redis Password: dlp
Redis Port: 6379
Redis Web Port: 8001


C:\Users\20111\AppData\Local\Temp\ipykernel_14980\4259667146.py:37: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


질문 1: Redis란 무엇인가요?
답변 1:  Redis는 오픈 소스 데이터베이스 시스템으로서, 메모리 내에서 데이터를 저장하고 접근할 수 있는 기능을 제공합니다. 주로 캐싱, 세션 관리, 메시지 브로커 등의 용도로 사용됩니다. 데이터를 디스크에도 저장할 수 있어서 영속성을 제공하며, 다양한 데이터 구조를 지원하여 복잡한 작업을 간단하게 처리할 수 있습니다. Redis는 빠른 속도와 간편한 사용성으로 널리 사용되고 있습니다.

질문 2: LangChain이 무엇인가요?
답변 2: 죄송합니다. LangChain에 대해서는 알려드릴 수 없습니다.

질문 3: Python은 무엇에 사용되나요?
답변 3: Python은 다양한 용도로 사용되는 프로그래밍 언어입니다. 웹 개발, 데이터 분석, 인공지능, 자동화, 게임 개발 등 다양한 분야에서 사용됩니다. 파이썬은 문법이 간결하고 읽기 쉬워서 입문자들이 쉽게 배울 수 있으며, 다양한 라이브러리와 프레임워크를 지원하여 빠르고 효율적인 개발이 가능합니다.

질문 4: 인공지능의 주요 응용 분야는 무엇인가요?
답변 4: 인공지능의 주요 응용 분야는 의료, 금융, 마케팅, 제조 및 로봇 공학 등이 있습니다. 의료 분야에서는 질병 진닝, 의료 이미지 분석, 약물 개발 등에 사용됩니다. 금융 분야에서는 사기 탐지, 자산 관리, 신용평가 등에 활용됩니다. 마케팅에서는 고객 행동 예측, 개인화된 광고 등에 사용됩니다. 제조 및 로봇 공학 분야에서는 자동화, 로봇 제어, 생산 최적화 등에 활용됩니다.

질문 5: Redis의 TTL 기능이란 무엇인가요?
답변 5: Redis의 TTL(Time-To-Live) 기능은 Redis 데이터베이스에서 특정 키의 수명을 설정하는 기능입니다. TTL 값을 설정하면 해당 키는 설정된 시간이 지난 후 자동으로 삭제됩니다. 이를 통해 데이터의 만료 및 자동 정리가 가능해지며, 캐시나 세션 관리 등에 유용하게 활용됩니다.

질문 6: LangChain의 Memory는 어떤 역할을 하나요?
답변 6: 죄송합

In [14]:
import time

# 데이터를 Redis에 저장
add_to_short_term_memory("질문1", "답변1")

# 30초 대기
time.sleep(30)

# 30초 단기 기억 확인
short_term_memory = load_short_term_memory()

if short_term_memory:
    print("단기 기억 내용:", short_term_memory)
else:
    print("단기 기억이 사라졌습니다.")


단기 기억이 사라졌습니다.


In [15]:
# Redis 서버 재시작 후 다시 확인
short_term_memory = load_short_term_memory()

if short_term_memory:
    print("단기 기억 내용:", short_term_memory)
else:
    print("단기 기억이 사라졌습니다.")


단기 기억이 사라졌습니다.
